In [ ]:
# Import dependencies
import numpy as np
import omnipose

# set up plotting defaults
from omnipose.plot import imshow
omnipose.plot.setup()

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
from omnipose.gpu import use_gpu
# use_GPU = use_gpu()

In [ ]:
# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
from glob import glob

files = list(sorted(glob("frames/*.png")))

In [ ]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99
imgs = [io.imread(f) for f in files]

# fig = plt.figure(figsize=[40]*2,frameon=False) # initialize figure

for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    if len(img.shape)>2:
        imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 
      

    imgs[k] = normalize99(imgs[k])

imgs = np.array(imgs)

In [ ]:
import cellpose_omni
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES

In [ ]:
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=False, model_type=model_name)

In [ ]:
import time
chans = [0,0] #this means segment based on first channel, no second channel 


# define parameters
params = {
    'channels':chans, # always define this with the model
    'rescale': None, # upscale or downscale your images, None = no rescaling 
    'mask_threshold': 3, # erode or dilate masks with higher or lower values between -5 and 5 
    'flow_threshold': 0, # default is .4, but only needed if there are spurious masks to clean up; slows down output
    'transparency': True, # transparency in flow output
    'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
    'cluster': True, # use DBSCAN clustering
    'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
    'verbose': False, # turn on if you want to see more output 
    'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
    'niter': None, # default None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
    'augment': False, # Can optionally rotate the image and average network outputs, usually not needed 
    # 'affinity_seg': True, # new feature, stay tuned...
}

tic = time.time() 
masks, flows, styles = model.eval(imgs,**params)

net_time = time.time() - tic
print('total segmentation time: {}s'.format(net_time))

In [ ]:
import cv2 as cv

for filename, mask in zip(files, masks):
    number = str(Path(filename).stem).split(".png")[0]
    cv.imwrite(f"frames/{number}-mask.png", mask)
    np.savetxt(f"frames/{number}-markers.csv", mask, delimiter=",")